# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 10 2023 4:44PM,259131,22,03132023,FUJIFILM Diosynth Biotechnologies Texas LLC,Released
1,Mar 10 2023 2:58PM,259121,10,MSP219423,Methapharm-G,Released
2,Mar 10 2023 2:34PM,259113,10,MSP218703-BO,Methapharm-G,Released
3,Mar 10 2023 2:04PM,259106,10,MSP219175,Methapharm-G,Released
4,Mar 10 2023 1:35PM,259102,19,9252947,VITABIOTICS USA INC.,Executing
5,Mar 10 2023 1:35PM,259102,19,9252948,VITABIOTICS USA INC.,Released
6,Mar 10 2023 1:35PM,259102,19,9252950,VITABIOTICS USA INC.,Released
7,Mar 10 2023 12:34PM,259091,12,7688010,Hush Hush,Executing
8,Mar 10 2023 12:00PM,259084,18,CAP-33995892,Capsugel,Released
9,Mar 10 2023 10:01AM,259071,21,RHL031023,"NBTY Global, Inc.",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
4,259102,Released,2
5,259106,Released,1
6,259113,Released,1
7,259121,Released,1
8,259131,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
259102,1.0,2.0
259106,NaN,1.0
259113,NaN,1.0
259121,NaN,1.0
259131,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
259071,1.0,0.0
259084,0.0,1.0
259091,1.0,0.0
259102,1.0,2.0
259106,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
259071,1,0
259084,0,1
259091,1,0
259102,1,2
259106,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,259071,1,0
1,259084,0,1
2,259091,1,0
3,259102,1,2
4,259106,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,259071,1,
1,259084,,1
2,259091,1,
3,259102,1,2
4,259106,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 10 2023 4:44PM,259131,22,FUJIFILM Diosynth Biotechnologies Texas LLC
1,Mar 10 2023 2:58PM,259121,10,Methapharm-G
2,Mar 10 2023 2:34PM,259113,10,Methapharm-G
3,Mar 10 2023 2:04PM,259106,10,Methapharm-G
4,Mar 10 2023 1:35PM,259102,19,VITABIOTICS USA INC.
7,Mar 10 2023 12:34PM,259091,12,Hush Hush
8,Mar 10 2023 12:00PM,259084,18,Capsugel
9,Mar 10 2023 10:01AM,259071,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 10 2023 4:44PM,259131,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,1
1,Mar 10 2023 2:58PM,259121,10,Methapharm-G,,1
2,Mar 10 2023 2:34PM,259113,10,Methapharm-G,,1
3,Mar 10 2023 2:04PM,259106,10,Methapharm-G,,1
4,Mar 10 2023 1:35PM,259102,19,VITABIOTICS USA INC.,1,2
5,Mar 10 2023 12:34PM,259091,12,Hush Hush,1,
6,Mar 10 2023 12:00PM,259084,18,Capsugel,,1
7,Mar 10 2023 10:01AM,259071,21,"NBTY Global, Inc.",1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 10 2023 4:44PM,259131,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,
1,Mar 10 2023 2:58PM,259121,10,Methapharm-G,1,
2,Mar 10 2023 2:34PM,259113,10,Methapharm-G,1,
3,Mar 10 2023 2:04PM,259106,10,Methapharm-G,1,
4,Mar 10 2023 1:35PM,259102,19,VITABIOTICS USA INC.,2,1
5,Mar 10 2023 12:34PM,259091,12,Hush Hush,,1
6,Mar 10 2023 12:00PM,259084,18,Capsugel,1,
7,Mar 10 2023 10:01AM,259071,21,"NBTY Global, Inc.",,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 10 2023 4:44PM,259131,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,
1,Mar 10 2023 2:58PM,259121,10,Methapharm-G,1,
2,Mar 10 2023 2:34PM,259113,10,Methapharm-G,1,
3,Mar 10 2023 2:04PM,259106,10,Methapharm-G,1,
4,Mar 10 2023 1:35PM,259102,19,VITABIOTICS USA INC.,2,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 10 2023 4:44PM,259131,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,NaN
1,Mar 10 2023 2:58PM,259121,10,Methapharm-G,1.0,NaN
2,Mar 10 2023 2:34PM,259113,10,Methapharm-G,1.0,NaN
3,Mar 10 2023 2:04PM,259106,10,Methapharm-G,1.0,NaN
4,Mar 10 2023 1:35PM,259102,19,VITABIOTICS USA INC.,2.0,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 10 2023 4:44PM,259131,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,0.0
1,Mar 10 2023 2:58PM,259121,10,Methapharm-G,1.0,0.0
2,Mar 10 2023 2:34PM,259113,10,Methapharm-G,1.0,0.0
3,Mar 10 2023 2:04PM,259106,10,Methapharm-G,1.0,0.0
4,Mar 10 2023 1:35PM,259102,19,VITABIOTICS USA INC.,2.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,777340,3.0,0.0
12,259091,0.0,1.0
18,259084,1.0,0.0
19,259102,2.0,1.0
21,259071,0.0,1.0
22,259131,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,777340,3.0,0.0
1,12,259091,0.0,1.0
2,18,259084,1.0,0.0
3,19,259102,2.0,1.0
4,21,259071,0.0,1.0
5,22,259131,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,3.0,0.0
1,12,0.0,1.0
2,18,1.0,0.0
3,19,2.0,1.0
4,21,0.0,1.0
5,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,3.0
1,12,Released,0.0
2,18,Released,1.0
3,19,Released,2.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,18,19,21,22
Status,,,,,,
Executing,0.0,1.0,0.0,1.0,1.0,0.0
Released,3.0,0.0,1.0,2.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,18,19,21,22
0,Executing,0.0,1.0,0.0,1.0,1.0,0.0
1,Released,3.0,0.0,1.0,2.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,18,19,21,22
0,Executing,0.0,1.0,0.0,1.0,1.0,0.0
1,Released,3.0,0.0,1.0,2.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()